### Categorical Feature Encoding Challenge
#### Нурдолотова С. 20930

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_auc_score
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')


In [2]:
# Dropping target and id and select the categorical columns

y = data_train['target']
data_id = data_test['id']

data_train=data_train.drop(['id','target'],axis=1)
data_test=data_test.drop(['id'],axis=1)

cate_cols = [cols for cols in data_train.columns if data_train[cols].dtype == 'object']

In [3]:
# Label Encoding the categorical columns 

encoder = LabelEncoder()
for col in cate_cols:
    data_train[col] = pd.DataFrame(encoder.fit_transform(data_train[col]))
    data_test[col] = pd.DataFrame(encoder.fit_transform(data_test[col]))   
x_train,x_valid,y_train,y_valid = train_test_split(data_train,y,random_state=1)

In [9]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Describing our model with 200 estimators and 2 scaled position weight (For imbalanced data)

clf = XGBClassifier(n_estimators=200,scale_pos_weight=2,random_state=1,colsample_bytree=0.5)
clf.fit(x_train,y_train)

C:\cond\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:40:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=2,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [5]:
predictions = clf.predict_proba(x_valid)[:,1]

In [6]:
# Calculating the score using roc_auc_score
score = roc_auc_score(y_valid,predictions)
print(score)

0.7730203319682851


In [7]:
predict = clf.predict_proba(data_test)[:,1]

In [8]:
submission = pd.DataFrame({'id': data_id, 'target': predict})
submission.to_csv('submission.csv', index=False)

submission

,id,target
0,300000,0.625434
1,300001,0.884096
2,300002,0.428617
3,300003,0.614134
4,300004,0.920911
...,...,...
199995,499995,0.430171
199996,499996,0.312330
199997,499997,0.323995
199998,499998,0.718053
